# Language Detection in MP3s using Librosa and Keras
* Noah Sragow
* David Haase

## Import Libraries

In [5]:
# !pip install librosa
# !pip install pydub

In [6]:
import pandas as pd
import numpy as np

import librosa

from audioutils import AudioSplitter, greek, arabic, turkish, persian, russian

source_languages = [arabic, greek, persian, russian, turkish]
ASML = AudioSplitter(source_languages)

target_dir = '/Volumes/LaCie Rose Gold 2TB/Datasets/Audio/'
seconds = 20
train_dir, test_dir = ASML.split(write_dir=target_dir, duration_s=seconds, test_split=.1)

2531 files already found in /Volumes/LaCie Rose Gold 2TB/Datasets/Audio/20_Seconds/Train_20/
Not rebuilding segments. Pass rebuild=True to force rebuild


## Load Data
* Audio files split into 20-second extracts
* 5,256 extracts created

In [2]:
def extract_linear_features(file_path):
    num_features = 40
    res_type = 'kaiser_fast'

    # handle exception to check if there isn't a file which is corrupted
    try:
      # here kaiser_fast is a technique used for faster extraction
        X, sample_rate = librosa.load(file_path, res_type=res_type)
      # we extract mfcc feature from data
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=num_features).T,axis=0)
    except Exception as e:
        print(e, "Error encountered while parsing file: ", file_path)
        return None

    return mfccs

In [3]:
write_dir = '/Volumes/LaCie Rose Gold 2TB/Datasets/Features/'
test_dir  = '/Volumes/LaCie Rose Gold 2TB/Datasets/Audio/20_Seconds/My_Test/'
kwargs = {'train_dir':train_dir, 'test_dir':test_dir, 'write_dir':write_dir, 'func':extract_linear_features}

ASML.extract_features(**kwargs)

.DS_Store: invalid audio filename; skipping


### Predict Features

In [4]:
test_file = '/Volumes/LaCie Rose Gold 2TB/Datasets/Features/feat_006/Test_Feature_Pickles/extracted.pkl'
df = pd.read_pickle(test_file)
df.head()

,File_ID,Filename,Lang_ID,Language,Path,Features
1,001,ar001.m4a,ar,Arabic,/Volumes/LaCie Rose Gold 2TB/Datasets/Audio/20...,"[-250.00165363974557, 132.40688140842465, -6.8..."
2,003,fa003.m4a,fa,Persian,/Volumes/LaCie Rose Gold 2TB/Datasets/Audio/20...,"[-240.55078909718804, 126.96836290678822, 1.79..."
3,004,gk004.m4a,gk,Greek,/Volumes/LaCie Rose Gold 2TB/Datasets/Audio/20...,"[-264.7438283356297, 107.12165049891314, 6.593..."
4,005,ru005.m4a,ru,Russian,/Volumes/LaCie Rose Gold 2TB/Datasets/Audio/20...,"[-257.1900915041211, 97.72281232550118, 3.7058..."
5,002,tr002.m4a,tr,Turkish,/Volumes/LaCie Rose Gold 2TB/Datasets/Audio/20...,"[-258.4246814083206, 107.15337234070702, 12.48..."
